# Creating fake 4LAC

First, I use this that I found on stack exchange just so that I can  import IceCubeAnalysis as a notebook

In [1]:


import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = current.read(f, 'json')


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.worksheets[0].cells:
                if cell.cell_type == 'code' and cell.language == 'python':
                # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.input)
                # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

/software/python-anaconda-2020.02-el7-x86_64/lib/python3.7/site-packages/nbformat/current.py:19: UserWarning: nbformat.current is deprecated.

- use nbformat for read/write/validate public API
- use nbformat.vX directly to composing notebooks of a particular version

  """


In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import IceCubeAnalysis

Next I load the real 4lac and icecube data. 

In [2]:

#Load all the old data in order to create fake data of these distributions
catalog_file_name =  "./processed_data/4LAC_catelogy.npz"
weights_type = 'flat'
source_class_names = ['BLL', 'bll', 'FSRQ', 'fsrq', 'BCU', 'bcu','RDG', 'rdg', 'AGN', 'agn', 'NLSY1', 'nlsy1','CSS','css','SSRQ','ssrq','SEY','sey']
icecube_file_name = "./processed_data/output_icecube_data.npz"
background_file_name = "./processed_data/output_icecube_background_count.npz"


#Set variables for integration
T = (10.0 * 365.25 * 24.0 * 3600.0)
E1 = 100.0
E2 = 30.0
alpha = 2.5
Aeff_filename = "./processed_data/output_icecube_AffIntegrated_%s.npz" % alpha

#Process Aeff and catelogue
sourcesearch_ = IceCubeAnalysis.SourceSearch(icecube_file_name)
sourcesearch_.load_background(background_file_name)

class_search = IceCubeAnalysis.SourceClassSearch(T, E1, E2, alpha, sourcesearch_, Aeff_filename)
class_search.load_4lac(catalog_file_name, source_class_names, weights_type)


Then I set the variables to be saved as the fake data to be with the same distribution as the real 4lac data. I set cat_names and cat_type all to the same thing for simplicity right now. Eventually I want to separate by blazar and nonblazar. I also need to edit out the milky way! 

In [3]:
cat_ra = np.random.choice(class_search.cat_ra,class_search.N)
cat_dec= np.random.choice(class_search.cat_dec,class_search.N)
cat_names = np.zeros(class_search.N).tolist()
cat_type = np.zeros(class_search.N).tolist()
cat_flux1000 = class_search.cat_flux1000
cat_z = np.random.choice(class_search.cat_z,class_search.N)
cat_var_index = np.random.choice(class_search.cat_var_index,class_search.N)

for i in range(class_search.N):
    cat_names[i] = "hello"
    cat_type[i] = 'agn'


Here I save all the fake data to a file, then I randomly select the coordinates of 10 fake sources and save them. This is so that I can eventually assign fake neutrino signals to these sources.  

In [4]:
np.savez("./processed_data/fake_4lac_set.npz",
         cat_names=cat_names,
         cat_ra=cat_ra,
         cat_dec=cat_dec,
         cat_type=cat_type,
         cat_flux1000=cat_flux1000,
         cat_z=cat_z,
         cat_var_index=cat_var_index)

#Randomly select 10 fake sources to place fake neutrinos at. Save this file. 
specific_ra = np.zeros(10)
specific_dec = np.zeros(10)
for i in range(10):
    index = np.random.randint(len(cat_ra),size=1)
    specific_ra[i] = cat_ra[index] 
    specific_dec[i] = cat_dec[index]
np.savez("./processed_data/specific_coords.npz",
         specific_ra = specific_ra,
         specific_dec = specific_dec)